In [1]:
# ____ hitting all question in dataset ____ 
import pandas as pd
import numpy as np
import os

# get path to data C:\Users\cayde\Desktop\Grad_School_stuff\data-534_expo_proj\Synergistic_Computing_hf\benchmarking\new\dataset\MATH\test
path = r'C:\Users\cayde\Desktop\Grad_School_stuff\data-534_expo_proj\Synergistic_Computing_hf\benchmarking\new\dataset\MATH\test'

# get list of the sub directorys in directory
sub_dir = os.listdir(path)

# create a dataframe to store the contents of the files
dataset = pd.DataFrame(columns=['problem', 'solution'])

# read in the contents of 71 files from each sub directory and print the contents
for i in sub_dir:
    # get path to sub directory
    sub_dir_path = os.path.join(path, i)
    # get list of files in sub directory
    files = os.listdir(sub_dir_path)
    # get path to each file in sub directory. only read in the first 71 files
    for j in files[:2]:
        file_path = os.path.join(sub_dir_path, j)
        # the contents of the file is a json object and i only want the 'problem' and 'solution' keys
        df = pd.read_json(file_path, orient='index', typ='series', encoding='utf-8')
        # append the contents of the file to the dataframe
        dataset = dataset.append(df[['problem', 'solution']], ignore_index=True)

# how many questions are in the dataset
print(len(dataset))

# # keep only first 2 rows of dataframe
# dataset = dataset[:2]


14


C:\Users\cayde\AppData\Local\Temp\ipykernel_16300\1383010087.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset = dataset.append(df[['problem', 'solution']], ignore_index=True)
C:\Users\cayde\AppData\Local\Temp\ipykernel_16300\1383010087.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset = dataset.append(df[['problem', 'solution']], ignore_index=True)
C:\Users\cayde\AppData\Local\Temp\ipykernel_16300\1383010087.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset = dataset.append(df[['problem', 'solution']], ignore_index=True)
C:\Users\cayde\AppData\Local\Temp\ipykernel_16300\1383010087.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version

In [2]:
# # ____ creating a prompt for the agent to answer ____

# # read in this json file C:\Users\cayde\Desktop\Grad_School_stuff\data-534_expo_proj\Synergistic_Computing_hf\benchmarking\new\dataset\MATH\test\algebra\1.json
# df = pd.read_json(r'C:\Users\cayde\Desktop\Grad_School_stuff\data-534_expo_proj\Synergistic_Computing_hf\benchmarking\new\dataset\MATH\test\algebra\1.json', orient='index', typ='series', encoding='utf-8')

# example_problem = df['problem']
# print(example_problem)
# # # create a prompt to send to agent. the prompt should explain that the agent is taking a test and that the agent should answer the question. and note the the equations are written in latex. the prompt should also contain the problem
# # prompt = 'You are taking a test. Answer the math questions to the best of your ability. Equations are written in latex. \n\n' + df['problem'] + '\n\n'

# # print(prompt)

In [3]:
# ____ get openai api key ____
# Path: C:\Users\cayde\Desktop\Grad_School_stuff\data-534_expo_proj\gpt-4-stuff\openai_key.txt
# open the file and read in the key
with open(r'C:\Users\cayde\Desktop\Grad_School_stuff\data-534_expo_proj\gpt-4-stuff\openai_key.txt', 'r') as f:
    openai_api_key = f.read()


In [4]:
# ____ init agent ____
# init langchain agent with gpt-4
from langchain import LLMChain, PromptTemplate, LLMMathChain
from langchain.llms import OpenAI
from langchain.agents import load_tools, initialize_agent, ZeroShotAgent, Tool, AgentExecutor
from langchain.chat_models import ChatOpenAI
from langchain.utilities import PythonREPL
from langchain.utilities.wolfram_alpha import WolframAlphaAPIWrapper
from langchain.chains import ConversationChain


llm = ChatOpenAI(model='gpt-4',temperature=0, openai_api_key=openai_api_key)
wolfram = WolframAlphaAPIWrapper(wolfram_alpha_appid='LXATGL-RJ6P2UWYT4')
python_repl = PythonREPL()

# ____ init tools ____
llm1 = OpenAI(temperature=0, openai_api_key=openai_api_key)
llm_math_chain = LLMMathChain(llm=llm1, verbose=True)
tools = [
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer simple questions about math"
    ),
    Tool(
        name="python_repl", 
        func=python_repl.run,
        description="A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`."
    ),
    Tool(
        name="wolfram_alpha",
        func=wolfram.run,
        description="A tool that can answer questions about math, science, and more. Input should be a question. For example, `What is the derivative of x^2?`"
    )
]

# ____ init prompt ____
prefix = """You are taking a test. Answer the math questions to the best of your ability. Equations are written in latex. You have access to the following tools:"""
suffix = """Begin! Remeber answer the math questions to the best of your ability. Equations are written in latex.

Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools, 
    prefix=prefix, 
    suffix=suffix, 
    input_variables=["input", "agent_scratchpad"]
)

# ____ run agent ____
llm_chain = LLMChain(llm=llm, prompt=prompt)

tool_names = [tool.name for tool in tools]
agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)

intermediate_steps = []
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True, return_intermediate_steps=True, max_iterations=7)

In [5]:
# # test agent is working
# response = agent_executor({'input':"what is 1+1?"})

# # how to grab the agents intermediate steps
# print(response["intermediate_steps"])

In [6]:
# ____ init gpt-4 ____
import openai

def ask_gpt4_student(prompt):
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {
                "role": "system",
                "content": """You are taking a test. Answer the math questions to the best of your ability. Equations are written in latex.""",
            },
            {
                "role": "user",
                "content": prompt,
            },
        ],
    )
    assistant_reply = response["choices"][0]["message"]["content"]
    return assistant_reply

def ask_gpt4_grader(question, student_answer, actual_answer):
    prompt = "question: " + question + '\nstudent_answer: ' + student_answer + '\nactual_answer: ' + actual_answer


    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {
                "role": "system",
                "content": """You are evaluating a student's work. you are provided the question the student was asked, and the student's work and answer, as well as the actual answer to the question. You are to evaluate the student's work and determine if the student was CORRECT or INCORRECT. Start each response with either CORRECT or INCORRECT explain briefly why the student is right or wrong. Equations are written in latex. """,
            },
            {
                "role": "user",
                "content": prompt,
            },
        ],
    )
    assistant_reply = response["choices"][0]["message"]["content"]
    return assistant_reply

In [7]:
# # test gpt-4 is working
# response = ask_gpt4("what is 1+1?")
# print(response)

In [8]:
# # test gpt-4 grader is working
# response = ask_gpt4_grader("what is 1+1?", "1+1=2", "2")
# print(response)

In [9]:
# add to 'dataset' dataframe add empty columns for agents intermediate steps and the agents answer. also add a column called gpt-4 answer.
dataset['agent_intermediate_steps'] = ''
dataset['agent_answer'] = ''
dataset['agents_score'] = ''
dataset['gpt-4_answer'] = 'failed to answer'
dataset['gpt-4_score'] = ''

In [10]:
# ____ run agent on dataset ____
for i in range(len(dataset)):
    # get problem from dataset
    problem = dataset['problem'][i]
    # get solution from dataset
    solution = dataset['solution'][i]
    try:
        # get the agents response
        response = agent_executor({'input':problem})
        # get the agents intermediate steps
        intermediate_steps = response["intermediate_steps"]
        # get the agents answer
        agent_answer = response["output"]
        # format agent answer and work for grading
        agent_intermediate_steps = str(dataset['agent_intermediate_steps'][i])
        agent_answer = str(dataset['agent_answer'][i])
        agent_scratchpad = agent_intermediate_steps + agent_answer
        # grade the agents answer
        agents_score = ask_gpt4_grader(problem, agent_scratchpad, solution)
    except Exception as e:
        agents_score = 'failed to answer'
        print(f"Error with agent: {e}")

    # get the gpt-4 answer
    gpt_4_answer = ask_gpt4_student(problem)
    # grade the gpt-4 answer
    gpt_4_score = ask_gpt4_grader(problem, gpt_4_answer, solution)
    # add the agents intermediate steps to the dataset
    dataset['agent_intermediate_steps'][i] = intermediate_steps
    # add the agents answer to the dataset
    dataset['agent_answer'][i] = agent_answer
    # add the gpt-4 answer to the dataset
    dataset['gpt-4_answer'][i] = gpt_4_answer
    # add the agents score to the dataset
    dataset['agents_score'][i] = agents_score
    # add the gpt-4 score to the dataset
    dataset['gpt-4_score'][i] = gpt_4_score



> Entering new AgentExecutor chain...
Thought: To find the vertical asymptotes, I need to find the values of x that make the denominator equal to zero.
Action: wolfram_alpha
Action Input: Solve x^2 + x - 6 = 0 for x
Observation: Assumption: solve x^2 + x - 6 = 0 for x 
Answer: x = -3
Thought:I found one value of x that makes the denominator equal to zero. I need to check if there are any other values.
Action: wolfram_alpha
Action Input: Are there any other solutions for x^2 + x - 6 = 0?
Observation: Wolfram Alpha wasn't able to answer it
Thought:I should try factoring the quadratic equation to find any other solutions.
Action: wolfram_alpha
Action Input: Factor x^2 + x - 6
Observation: Assumption: factor | x^2 + x - 6 
Answer: (x + 3) (x - 2)
Thought:I now know the factors of the quadratic equation, which will help me find the other value of x that makes the denominator equal to zero.
Action: wolfram_alpha
Action Input: Solve (x + 3)(x - 2) = 0 for x
Observation: Assumption: solve (x

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: The server had an error while processing your request. Sorry about that!.


Now I have the length of side BC, which is 5.4. I can use the Pythagorean theorem to find the length of side AC. The formula is: AC^2 = AB^2 - BC^2.
Action: Calculator
Action Input: (9**2) - (5.4**2)

> Entering new LLMMathChain chain...
(9**2) - (5.4**2)Answer: 58.24
> Finished chain.

Observation: Answer: 58.24
Thought:Now I have the value of AC^2, which is 58.24. I need to find the square root of this value to get the length of side AC.
Action: Calculator
Action Input: sqrt(58.24)

> Entering new LLMMathChain chain...
sqrt(58.24)
```python
import math
print(math.sqrt(58.24))
```

Answer: 7.631513611335565

> Finished chain.

Observation: Answer: 7.631513611335565

Thought:Now I have the length of side AC, which is approximately 7.63. I can use the cosine formula to find the cosine of angle C. The formula is: cos(C) = (AC^2 + BC^2 - AB^2) / (2 * AC * BC).
Action: Calculator
Action Input: ((7.63**2) + (5.4**2) - (9**2)) / (2 * 7.63 * 5.4)

> Entering new LLMMathChain chain...
((7.63**

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: The server had an error while processing your request. Sorry about that!.


Thought: I need to find the number of two-digit multiples of 45 and divide it by the total number of multiples of 45 less than 1000.
Action: Calculator
Action Input: 1000 / 45

> Entering new LLMMathChain chain...
1000 / 45Answer: 22.22222222222222
> Finished chain.

Observation: Answer: 22.22222222222222
Thought:Error with agent: Could not parse LLM output: `There are 22 multiples of 45 less than 1000, but I need to find how many of them are two-digit integers.`


> Entering new AgentExecutor chain...
Thought: First, I need to convert the base 5 and base 8 numbers to base 10.
Action: Calculator
Action Input: int("2425", 5)

> Entering new LLMMathChain chain...
int("2425", 5)Answer: 474
> Finished chain.

Observation: Answer: 474
Thought:I have converted the base 5 number to base 10. Now, I need to convert the base 8 number to base 10.
Action: Calculator
Action Input: int("367", 8)

> Entering new LLMMathChain chain...
int("367", 8)Answer: 247
> Finished chain.

Observation: Answer: 24

In [11]:
# ____ save dataset ____
# save dataset to cwd
dataset.to_csv('20_question.csv', index=False)